In [1]:
import os
import time
import torch
from torchvision.io import read_image
from torchvision.models.detection import fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
from tqdm import tqdm

In [2]:
coco_annotation_path = "../datasets/coco128/annotations/instances_train2017v2.json"
image_paths = "../datasets/coco128/images/train2017"
temp_folder = "temp"

os.makedirs(temp_folder, exist_ok=True)

coco_gt = COCO(coco_annotation_path)
image_ids = list(coco_gt.imgs.keys())

weights = FCOS_ResNet50_FPN_Weights.DEFAULT
model = fcos_resnet50_fpn(weights=weights)
model.eval()

preprocess = weights.transforms()
threshold = 0.45

model_to_coco_category = {}
model_categories = weights.meta["categories"]
for coco_cat_id, cat_info in coco_gt.cats.items():
    coco_cat_name = cat_info['name']
    for model_cat_id, model_cat_name in enumerate(model_categories):
        if coco_cat_name.lower() == model_cat_name.lower():
            model_to_coco_category[model_cat_id] = coco_cat_id

coco_results = []

start_time = time.time()

for img_id in tqdm(image_ids):
    img_info = coco_gt.imgs[img_id]
    file_name = img_info['file_name']
    image_path = os.path.join(image_paths, file_name)

    img = read_image(image_path)

    try:
        if img.shape[0] == 1:
            img = img.repeat(3, 1, 1)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        continue
    
    processed = preprocess(img)
    
    with torch.no_grad():
        prediction = model([processed])[0]
    
    keep = prediction['scores'] > threshold
    boxes = prediction['boxes'][keep]
    scores = prediction['scores'][keep]
    labels = prediction['labels'][keep]
    
    for box, score, label in zip(boxes, scores, labels):
        model_label = label.item()
        
        if model_label not in model_to_coco_category:
            continue
            
        coco_label = model_to_coco_category[model_label]
        
        x1, y1, x2, y2 = box.tolist()
        coco_box = [x1, y1, x2 - x1, y2 - y1]
        
        result = {
            'image_id': img_id,
            'category_id': coco_label,
            'bbox': coco_box,
            'score': score.item()
        }
        coco_results.append(result)

end_time = time.time()
total_time = end_time - start_time
average_time = total_time / len(image_ids)


results_file = os.path.join(temp_folder, "detection_results.json")
with open(results_file, 'w') as f:
    json.dump(coco_results, f)

try:
    coco_dt = coco_gt.loadRes(results_file)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    mAP50_95 = coco_eval.stats[0]
    mAP50 = coco_eval.stats[1]

    print(f"\nProcessed {len(image_ids)} images.")
    print(f"Total inference time: {total_time:.2f} seconds")
    print(f"Average inference time per image: {average_time:.2f} seconds")
    print(f"mAP @ IoU=0.50:0.95: {mAP50_95:.4f}")
    print(f"mAP @ IoU=0.50: {mAP50:.4f}")
except Exception as e:
    print(f"Error during evaluation: {e}")

if os.path.exists(results_file):
    os.remove(results_file)
try:
    os.rmdir(temp_folder)
except OSError:
    pass

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 128/128 [01:57<00:00,  1.09it/s]


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.522
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.711
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.584
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.273
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.552
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.706
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.559
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.566
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10